# Genetic Algorithm

In [ ]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [ ]:
creator.create('MaxMinFitness', base.Fitness, weights = (1.0, -1.0))
creator.create('Individual', list, fitness = creator.MaxMinFitness)

In [ ]:
def population(n, items_len, b_size = 2):
    return [creator.Individual(list(np.random.randint(2, size = items_len * b_size))) for _ in range(n)]

In [ ]:
def to_intarray(ind, b_size):
    x = np.around(np.array(ind).reshape((-1, b_size)))
    return x.dot(1 << np.arange(x.shape[-1])[::-1])

def eval_value(ind, items, b_size, max_weight, penalty):
    x = to_intarray(ind, b_size)
    
    v = sum(x * items[:, 0])
    w = sum(x * items[:, 1])
    
    return (v, w) if w <= max_weight else (0, penalty)

def create_toolbox(items, b_size = 2, max_weight = 65, penalty = 1000, indpb = 0.05, eta = 20.0):
    toolbox = base.Toolbox()

    toolbox.register('evaluate', eval_value, items = items, b_size = b_size, max_weight = max_weight, penalty = penalty)
    toolbox.register('mate', tools.cxSimulatedBinaryBounded, low = 0, up = 1, eta = eta)
    toolbox.register('mutate', tools.mutPolynomialBounded, low = 0, up = 1, eta = eta, indpb = indpb)
    toolbox.register('select', tools.selNSGA2)
    
    return toolbox

In [ ]:
def evolve(n, items, ngen = 10, b_size = 2, max_weight = 65, penalty = 1000, cxpb = 0.5, mutpb = 0.2, indpb = 0.05, eta = 20.0, child_rate = 1.0):
    toolbox = create_toolbox(items, b_size, max_weight, penalty, indpb, eta)
    
    p = population(n, len(items), b_size)
    
    mu = len(p)
    child_num = int(len(p) * child_rate)
    
    res, _ = algorithms.eaMuPlusLambda(p, toolbox, mu, lambda_ = child_num, ngen = ngen, cxpb = cxpb, mutpb = mutpb)
    
    return res

In [ ]:
def ga(n, items, ngen = 10, b_size = 2, max_weight = 65, penalty = 1000, cxpb = 0.5, mutpb = 0.2, indpb = 0.05, eta = 20.0, child_rate = 1.0):
    rs = evolve(n, items, ngen, b_size, max_weight, penalty, cxpb, mutpb, indpb, eta, child_rate)
    
    return to_intarray(tools.selBest(rs, 1)[0], b_size)

In [ ]:
items1 = np.array([
    [120, 10],
    [130, 12],
    [80, 7],
    [100, 9],
    [250, 21],
    [185, 16]
])

In [ ]:
ga(100, items1, ngen = 50)